In [1]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import skimage.io as io
import os, sys
from matplotlib import pyplot as plt


sys.path.append(os.path.join(os.path.join(os.getcwd(), "../../../../../tf-image-segmentation")))
from tf_image_segmentation.utils import set_paths # Sets appropriate paths and provides access to log_dir and checkpoint_path via FLAGS

FLAGS = set_paths.FLAGS

checkpoints_dir = FLAGS.checkpoints_dir
log_dir = os.path.join(FLAGS.log_dir + "deeplab/")

slim = tf.contrib.slim

os.environ["CUDA_VISIBLE_DEVICES"] = '1'

slim = tf.contrib.slim

from tf_image_segmentation.models.fcn_8s import FCN_8s

from tf_image_segmentation.models.resnet_v1_101_8s import resnet_v1_101_8s, extract_resnet_v1_101_mapping_without_logits

from matplotlib import pyplot as plt
from tf_image_segmentation.utils.pascal_voc import pascal_segmentation_lut
from tf_image_segmentation.utils.tf_records import read_tfrecord_and_decode_into_image_annotation_pair_tensors
from tf_image_segmentation.utils.inference import adapt_network_for_any_size_input
from tf_image_segmentation.utils.visualization import visualize_segmentation_adaptive

pascal_voc_lut = pascal_segmentation_lut()

tfrecord_filename = 'pascal_augmented_val.tfrecords'

number_of_classes = 21

filename_queue = tf.train.string_input_producer(
    [tfrecord_filename], num_epochs=1)

image, annotation = read_tfrecord_and_decode_into_image_annotation_pair_tensors(filename_queue)

# Fake batch for image and annotation by adding
# leading empty axis.
image_batch_tensor = tf.expand_dims(image, axis=0)
annotation_batch_tensor = tf.expand_dims(annotation, axis=0)

# Be careful: after adaptation, network returns final labels
# and not logits
resnet_v1_101_8s = adapt_network_for_any_size_input(resnet_v1_101_8s, 8)

is_training_ph = tf.placeholder(tf.bool)
pred, fcn_16s_variables_mapping = resnet_v1_101_8s(image_batch_tensor=image_batch_tensor,
                                                   number_of_classes=number_of_classes,
                                                   is_training=is_training_ph)

# Take away the masked out values from evaluation
weights = tf.to_float( tf.not_equal(annotation_batch_tensor, 255) )

# Define the accuracy metric: Mean Intersection Over Union
miou, update_op = slim.metrics.streaming_mean_iou(predictions=pred,
                                                   labels=annotation_batch_tensor,
                                                   num_classes=number_of_classes,
                                                   weights=weights)



In [2]:
# The op for initializing the variables.
initializer = tf.local_variables_initializer()

saver = tf.train.Saver()

with tf.Session() as sess:
    
    sess.run(initializer)

    saver.restore(sess, '/home/vittal/work/segmentation/tf-image-segmentation/tf_image_segmentation/recipes/pascal_voc/DeepLab/model_resnet_101_8s.ckpt')
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    # There are 904 images in restricted validation dataset
    for i in xrange(904):
        
        image_np, annotation_np, pred_np, tmp = sess.run([image, annotation, pred, update_op], feed_dict={is_training_ph: False})
        
        # Display the image and the segmentation result
        if i%100 == 0:
            print(i)
#         upsampled_predictions = pred_np.squeeze()
#         plt.imshow(image_np)
#         plt.show()
#         visualize_segmentation_adaptive(upsampled_predictions, pascal_voc_lut)
        
    coord.request_stop()
    coord.join(threads)
    
    res = sess.run(miou)
    
    print("Pascal VOC 2012 Restricted (RV-VOC12) Mean IU: " + str(res))

0
100
200
300
400
500
600
700
800
900
Pascal VOC 2012 Restricted (RV-VOC12) Mean IU: 0.651866
